In [14]:
ticker = "MSFT"   # Company selection

In [ ]:
# ------------------------------
# 1. Get company facts from SEC
# ------------------------------

import requests
import pandas as pd

HEADERS = {"User-Agent": "nelson@gmail.com"}  # replace with your email

def get_cik_from_ticker(ticker: str) -> str:
    """Return CIK (10-digit) from ticker symbol."""
    url = "https://www.sec.gov/files/company_tickers.json"
    resp = requests.get(url, headers=HEADERS)
    data = resp.json()
    for _, v in data.items():
        if v["ticker"].upper() == ticker.upper():
            return str(v["cik_str"]).zfill(10)
    raise ValueError(f"CIK not found for {ticker}")

def get_company_facts(ticker: str):
    """Download companyfacts JSON for a given ticker."""
    cik = get_cik_from_ticker(ticker)
    url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
    print(f"Company facts URL: {url}")
    resp = requests.get(url, headers=HEADERS)
    return resp.json()

# ------------------------------
# Use ticker defined previously
# ------------------------------
facts = get_company_facts(ticker)

# Get company name
company_name = facts.get("entityName", "Unknown")

# Try to detect last filing using Assets
try:
    assets = facts["facts"]["us-gaap"]["Assets"]["units"]["USD"]
    df_assets = pd.DataFrame(assets)
    df_assets = df_assets.sort_values("filed", ascending=False)
    latest_filed = df_assets.iloc[0][["form", "filed"]].to_dict()
except Exception:
    latest_filed = {"form": "N/A", "filed": "N/A"}

print(f"\nCompany: {company_name}")
print(f"Last Filing: Form {latest_filed['form']} filed on {latest_filed['filed']}")


Company facts URL: https://data.sec.gov/api/xbrl/companyfacts/CIK0000789019.json

Company: MICROSOFT CORPORATION
Last Filing: Form 10-K filed on 2025-07-30


In [ ]:
# ------------------------------
# 2. Balance Sheet Evolution (last 5 filings)
# ------------------------------

def extract_facts_table(facts, concept, unit="USD"):
    """Return DataFrame with fy, fp, filed, form, val for a given concept."""
    try:
        items = facts["facts"]["us-gaap"][concept]["units"][unit]
        df = pd.DataFrame(items)
        df = df[["fy", "fp", "filed", "form", "val"]].sort_values("filed", ascending=False)
        return df
    except KeyError:
        return pd.DataFrame()

# Extract series
assets_df = extract_facts_table(facts, "Assets")
liabilities_df = extract_facts_table(facts, "Liabilities")
equity_df = extract_facts_table(facts, "StockholdersEquity")

# Merge into one table
balance_df = assets_df.merge(
    liabilities_df, on=["fy","fp","filed","form"], how="outer", suffixes=("_assets","_liabilities")
).merge(
    equity_df, on=["fy","fp","filed","form"], how="outer"
)

# Rename columns for clarity
balance_df = balance_df.rename(columns={"val":"Equity"})
balance_df = balance_df.rename(columns={"val_assets":"Assets", "val_liabilities":"Liabilities"})

# Sort by filing date and keep last 5
balance_df = balance_df.sort_values("filed", ascending=False).head(5)

print("\n=== Balance Sheet Evolution (last 5 filings) ===")
print(balance_df.to_string(index=False))



=== Balance Sheet Evolution (last 5 filings) ===
    fy fp      filed form       Assets  Liabilities       Equity
2025.0 FY 2025-07-30 10-K 512163000000 2.436860e+11 268477000000
2025.0 FY 2025-07-30 10-K 619003000000 2.436860e+11 268477000000
2025.0 FY 2025-07-30 10-K 512163000000 2.436860e+11 206223000000
2025.0 FY 2025-07-30 10-K 619003000000 2.755240e+11 343479000000
2025.0 FY 2025-07-30 10-K 619003000000 2.755240e+11 268477000000


In [ ]:
# ------------------------------
# 3. Last Balance Sheet (table-style accounts, fixed with end date)
# ------------------------------

def extract_last_balance_accounts(facts, unit="USD"):
    """
    Extract detailed Assets and Liabilities accounts 
    for the last available filing, filtered by the correct 'end' date.
    """
    # Step 1: detect last reporting date from Total Assets
    assets_all = facts["facts"]["us-gaap"]["Assets"]["units"][unit]
    df_assets = pd.DataFrame(assets_all).sort_values("filed", ascending=False)
    
    # The most recent filing has two columns: current end date + comparative end date
    last_end = df_assets.iloc[0]["end"]   # most recent period
    last_filed, last_form = df_assets.iloc[0]["filed"], df_assets.iloc[0]["form"]

    # Step 2: accounts of interest (expanded)
    concepts_of_interest = [
        # Assets
        "CashAndCashEquivalentsAtCarryingValue",
        "MarketableSecuritiesCurrent",
        "AccountsReceivableNetCurrent",
        "InventoryNet",
        "OtherAssetsCurrent",
        "AssetsCurrent",
        "MarketableSecuritiesNoncurrent",
        "PropertyPlantAndEquipmentNet",
        "OtherAssetsNoncurrent",
        "AssetsNoncurrent",
        "Assets",
        # Liabilities
        "AccountsPayableCurrent",
        "OtherLiabilitiesCurrent",
        "DeferredRevenueCurrent",     
        "CommercialPaper",             
        "LiabilitiesCurrent",
        "LongTermDebtNoncurrent",
        "OtherLiabilitiesNoncurrent",
        "LiabilitiesNoncurrent",
        "Liabilities",
        "LiabilitiesAndStockholdersEquity"
    ]

    # Step 4: filter by same filed, form, and end date
    records = []
    for concept in concepts_of_interest:
        try:
            df = pd.DataFrame(facts["facts"]["us-gaap"][concept]["units"][unit])
            df_last = df[(df["filed"] == last_filed) & 
                         (df["form"] == last_form) & 
                         (df["end"] == last_end)]
            if not df_last.empty:
                row = df_last.iloc[0]
                records.append({
                    "Concept": concept,
                    "Value (USD)": row["val"],
                    "End Date": row["end"],
                    "Fiscal Year": row.get("fy"),
                    "Period": row.get("fp"),
                    "Filed": row.get("filed"),
                    "Form": row.get("form")
                })
        except KeyError:
            continue

    df_final = pd.DataFrame(records).sort_values("Concept")

    # Step 5: backup calculation for LiabilitiesNoncurrent
    if "LiabilitiesNoncurrent" not in df_final["Concept"].values:
        try:
            total_liabilities = df_final.loc[df_final["Concept"] == "Liabilities", "Value (USD)"].values[0]
            current_liabilities = df_final.loc[df_final["Concept"] == "LiabilitiesCurrent", "Value (USD)"].values[0]
            missing_val = total_liabilities - current_liabilities
            df_final = pd.concat([df_final, pd.DataFrame([{
                "Concept": "LiabilitiesNoncurrent",
                "Value (USD)": missing_val,
                "End Date": last_end,
                "Fiscal Year": df_final["Fiscal Year"].iloc[0],
                "Period": df_final["Period"].iloc[0],
                "Filed": last_filed,
                "Form": last_form
            }])], ignore_index=True)
        except Exception:
            pass

    return df_final.sort_values("Concept")

# ------------------------------
# Usage
# ------------------------------
last_balance_accounts = extract_last_balance_accounts(facts)

print("\n=== Last Balance Sheet (actual period only) ===")
print(last_balance_accounts.to_string(index=False))




=== Last Balance Sheet (actual period only) ===
                              Concept  Value (USD)   End Date  Fiscal Year Period      Filed Form
               AccountsPayableCurrent  27724000000 2025-06-30       2025.0     FY 2025-07-30 10-K
         AccountsReceivableNetCurrent  69905000000 2025-06-30       2025.0     FY 2025-07-30 10-K
                               Assets 619003000000 2025-06-30       2025.0     FY 2025-07-30 10-K
                        AssetsCurrent 191131000000 2025-06-30       2025.0     FY 2025-07-30 10-K
CashAndCashEquivalentsAtCarryingValue  30242000000 2025-06-30       2025.0     FY 2025-07-30 10-K
                      CommercialPaper            0 2025-06-30       2025.0     FY 2025-07-30 10-K
                         InventoryNet    938000000 2025-06-30       2025.0     FY 2025-07-30 10-K
                          Liabilities 275524000000 2025-06-30       2025.0     FY 2025-07-30 10-K
     LiabilitiesAndStockholdersEquity 619003000000 2025-06-30       2

In [25]:
# ------------------------------
# Financial Ratios from Balance Sheet
# ------------------------------

import pandas as pd

def calculate_ratios(balance_df):
    """
    Calculate 15 key financial ratios from the last balance sheet.
    balance_df must contain Concept and Value (USD) columns.
    """
    # Convert to dict for easy access
    data = dict(zip(balance_df["Concept"], balance_df["Value (USD)"]))
    
    # Extract main values (if not present, default 0)
    assets = data.get("Assets", 0)
    liabilities = data.get("Liabilities", 0)
    equity = assets - liabilities
    
    current_assets = data.get("AssetsCurrent", 0)
    current_liabilities = data.get("LiabilitiesCurrent", 0)
    cash = data.get("CashAndCashEquivalentsAtCarryingValue", 0)
    market_sec = data.get("MarketableSecuritiesCurrent", 0)
    receivables = data.get("AccountsReceivableNetCurrent", 0)
    inventories = data.get("InventoryNet", 0)
    ppe = data.get("PropertyPlantAndEquipmentNet", 0)
    long_term_debt = data.get("LongTermDebtNoncurrent", 0)
    noncurrent_liabilities = data.get("LiabilitiesNoncurrent", 0)

    ratios = {}

    # Liquidity
    ratios["Current Ratio"] = current_assets / current_liabilities if current_liabilities else None
    ratios["Quick Ratio"] = (current_assets - inventories) / current_liabilities if current_liabilities else None

    # Leverage / Solvency
    ratios["Debt to Assets"] = liabilities / assets if assets else None
    ratios["Debt to Equity"] = liabilities / equity if equity else None
    ratios["Equity Ratio"] = equity / assets if assets else None
    ratios["Long-Term Debt to Equity"] = long_term_debt / equity if equity else None

    # Working Capital
    ratios["Working Capital"] = current_assets - current_liabilities
    ratios["NWC / Assets"] = (current_assets - current_liabilities) / assets if assets else None

    # Debt Coverage
    ratios["Current Liabilities / Total Liabilities"] = current_liabilities / liabilities if liabilities else None
    ratios["Noncurrent Liabilities / Total Liabilities"] = noncurrent_liabilities / liabilities if liabilities else None

    # Asset Structure
    ratios["Cash & Equivalents / Assets"] = cash / assets if assets else None
    ratios["Receivables / Assets"] = receivables / assets if assets else None
    ratios["Inventories / Assets"] = inventories / assets if assets else None
    ratios["PPE / Assets"] = ppe / assets if assets else None

    return pd.Series(ratios)

# ------------------------------
# Example usage with your DataFrame
# ------------------------------

# Assume last_balance_accounts is your DataFrame with Concept + Value (USD)
ratios = calculate_ratios(last_balance_accounts)

print("\n=== Financial Ratios (15 indicators) ===")
print(ratios.to_string())



=== Financial Ratios (15 indicators) ===
Current Ratio                                 1.353446e+00
Quick Ratio                                   1.346804e+00
Debt to Assets                                4.451093e-01
Debt to Equity                                8.021568e-01
Equity Ratio                                  5.548907e-01
Long-Term Debt to Equity                      1.168980e-01
Working Capital                               4.991300e+10
NWC / Assets                                  8.063450e-02
Current Liabilities / Total Liabilities       5.125434e-01
Noncurrent Liabilities / Total Liabilities    4.874566e-01
Cash & Equivalents / Assets                   4.885598e-02
Receivables / Assets                          1.129316e-01
Inventories / Assets                          1.515340e-03
PPE / Assets                                  3.311228e-01


In [ ]:
# ------------------------------
# 6. Last 4 Income Statements
# ------------------------------

def extract_recent_income_statements(facts, concepts=("Revenues", "OperatingIncomeLoss", "NetIncomeLoss"), unit="USD", n=4):
    """
    Extract the last n income statements (revenues, operating income, net income).
    """
    records = []

    # Iterate over concepts and store all rows
    for concept in concepts:
        try:
            items = facts["facts"]["us-gaap"][concept]["units"][unit]
            df = pd.DataFrame(items)[["fy","fp","filed","form","end","val"]]
            df["Concept"] = concept
            records.append(df)
        except KeyError:
            continue

    if not records:
        return pd.DataFrame()

    # Combine all concepts
    all_data = pd.concat(records, ignore_index=True)

    # Get last n unique periods based on "end" date
    last_periods = (all_data.sort_values("end", ascending=False)
                             [["fy","fp","filed","form","end"]]
                             .drop_duplicates()
                             .head(n))

    # Filter only those periods
    filtered = pd.merge(all_data, last_periods, on=["fy","fp","filed","form","end"], how="inner")

    # Pivot to wide format (concepts as columns)
    result = filtered.pivot_table(index=["fy","fp","filed","form","end"],
                                  columns="Concept", values="val", aggfunc="first").reset_index()

    # Order by end date descending
    result = result.sort_values("end", ascending=False).reset_index(drop=True)

    return result

# ------------------------------
# Usage
# ------------------------------
income_df = extract_recent_income_statements(facts, n=4)
print("\n=== Last 4 Income Statements ===")
print(income_df.to_string(index=False))



=== Last 4 Income Statements ===
    fy fp      filed form        end  NetIncomeLoss  OperatingIncomeLoss
2025.0 FY 2025-07-30 10-K 2025-06-30   101832000000         128528000000
2025.0 Q3 2025-04-30 10-Q 2025-03-31    74599000000          94205000000
2025.0 Q2 2025-01-29 10-Q 2024-12-31    48775000000          62205000000
2025.0 Q1 2024-10-30 10-Q 2024-09-30    24667000000          30552000000


In [ ]:
# ------------------------------
# 6. Cash Flow Statement
# ------------------------------

def extract_cashflow(facts, unit="USD", n=4):
    """
    Extract last n available cash flow statements.
    """
    concepts = [
        "NetCashProvidedByUsedInOperatingActivities",
        "NetCashProvidedByUsedInInvestingActivities",
        "NetCashProvidedByUsedInFinancingActivities",
        "CashAndCashEquivalentsPeriodIncreaseDecrease"
    ]
    
    records = []

    for concept in concepts:
        try:
            items = facts["facts"]["us-gaap"][concept]["units"][unit]
            df = pd.DataFrame(items)[["fy","fp","filed","form","end","val"]]
            df["Concept"] = concept
            records.append(df)
        except KeyError:
            continue

    if not records:
        return pd.DataFrame()

    # Combine
    all_data = pd.concat(records, ignore_index=True)

    # Últimos n períodos únicos por fecha de cierre
    last_periods = (all_data.sort_values("end", ascending=False)
                             [["fy","fp","filed","form","end"]]
                             .drop_duplicates()
                             .head(n))

    # Filtrar
    filtered = pd.merge(all_data, last_periods, on=["fy","fp","filed","form","end"], how="inner")

    # Pivot table
    result = filtered.pivot_table(index=["fy","fp","filed","form","end"],
                                  columns="Concept", values="val", aggfunc="first").reset_index()

    result = result.sort_values("end", ascending=False).reset_index(drop=True)

    return result

# ------------------------------
# Usage
# ------------------------------
cashflow_df = extract_cashflow(facts, n=4)

print("\n=== Last Cash Flow Statements (up to 4) ===")
print(cashflow_df.to_string(index=False))



=== Last Cash Flow Statements (up to 4) ===
    fy fp      filed form        end  NetCashProvidedByUsedInFinancingActivities  NetCashProvidedByUsedInInvestingActivities  NetCashProvidedByUsedInOperatingActivities
2025.0 FY 2025-07-30 10-K 2025-06-30                                -51699000000                                -72599000000                                136162000000
2025.0 Q3 2025-04-30 10-Q 2025-03-31                                -40855000000                                -42027000000                                 93515000000
2025.0 Q2 2025-01-29 10-Q 2024-12-31                                -27819000000                                -29313000000                                 56471000000
2025.0 Q1 2024-10-30 10-Q 2024-09-30                                -16576000000                                -15201000000                                 34180000000
